In [55]:
import sys 
sys.path.append("../")
import pandas as pd 
import plotly.graph_objects as go 
import datetime as dt 
from plotting import CandlePlot

In [56]:
from infratstructure.instrument_collection import instrumentCollection as ic

In [57]:
pair = "USD_CHF"
granularity = "H4"
df = pd.read_pickle(f"../data/{pair}_{granularity}.pkl")
MA_LIST = [10, 20, 50, 200]

In [58]:
df.columns

Index(['time', 'volume', 'mid_o', 'mid_h', 'mid_l', 'mid_c', 'bid_o', 'bid_h',
       'bid_l', 'bid_c', 'ask_o', 'ask_h', 'ask_l', 'ask_c'],
      dtype='object')

In [59]:
df_ma = df[["time", "mid_o", "mid_h", "mid_l", "mid_c"]].copy()

In [60]:
df_ma.head()

,time,mid_o,mid_h,mid_l,mid_c
0,2022-10-13 01:00:00+00:00,0.99677,0.99832,0.99659,0.99764
1,2022-10-13 05:00:00+00:00,0.99761,1.00045,0.99732,0.99864
2,2022-10-13 09:00:00+00:00,0.99868,1.00710,0.99601,1.00454
3,2022-10-13 13:00:00+00:00,1.00455,1.00738,0.99697,0.99996
4,2022-10-13 17:00:00+00:00,0.99998,1.00136,0.99713,1.00122


In [61]:
for ma in MA_LIST:

    df_ma[f"MA_{ma}"] = df_ma.mid_c.rolling(window=ma).mean()
df_ma.dropna(inplace=True)
df_ma.reset_index(inplace=True, drop=True)


In [62]:
df_plot = df_ma.iloc[:500]

In [63]:
cp = CandlePlot(df_plot)

In [64]:
traces = [f"MA_{x}" for x in MA_LIST]
cp.show_plot(line_traces=traces)

In [65]:
MA_S = "MA_50"
MA_L = "MA_200"
BUY = 1 
SELL = -1 
NONE = 0

In [66]:
df_an = df_ma[["time", "mid_o", "mid_h", "mid_l", "mid_c", MA_S, MA_L]].copy()

In [67]:
df_an.head()

,time,mid_o,mid_h,mid_l,mid_c,MA_50,MA_200
0,2022-11-29 06:00:00+00:00,0.94633,0.95247,0.94626,0.95185,0.949444,0.978628
1,2022-11-29 10:00:00+00:00,0.95182,0.95396,0.94844,0.95272,0.949590,0.978403
2,2022-11-29 14:00:00+00:00,0.95270,0.95458,0.95011,0.95228,0.949688,0.978172
3,2022-11-29 18:00:00+00:00,0.95230,0.95452,0.95222,0.95391,0.949724,0.977918
4,2022-11-29 22:00:00+00:00,0.95200,0.95472,0.95200,0.95284,0.949725,0.977683


In [68]:
df_an["DELTA"] = df_an.MA_50 - df_an.MA_200
df_an["DELTA_PREV"] = df_an.DELTA.shift(1)

In [69]:
def is_trade(row):
    if row.DELTA >= 0 and row.DELTA_PREV < 0:
        return BUY
    elif row.DELTA < 0 and row.DELTA_PREV >= 0:
        return SELL
    return NONE
    

In [70]:
df_an["TRADE"] = df_an.apply(is_trade, axis=1)

In [71]:
df_trades = df_an[df_an.TRADE != NONE].copy()

In [72]:
df_trades.head()

,time,mid_o,mid_h,mid_l,mid_c,MA_50,MA_200,DELTA,DELTA_PREV,TRADE
349,2023-02-20 10:00:00+00:00,0.92380,0.92452,0.92226,0.92408,0.922784,0.922704,0.000080,-0.000016,1
464,2023-03-17 13:00:00+00:00,0.92792,0.92934,0.92638,0.92866,0.927109,0.927126,-0.000018,0.000073,-1
733,2023-05-19 09:00:00+00:00,0.90298,0.90366,0.90040,0.90188,0.895849,0.895780,0.000069,-0.000231,1
865,2023-06-20 09:00:00+00:00,0.89690,0.89840,0.89641,0.89750,0.900077,0.900154,-0.000077,0.000232,-1
1107,2023-08-15 17:00:00+00:00,0.87834,0.87890,0.87724,0.87849,0.875709,0.875557,0.000153,-0.000013,1


In [83]:
cp = CandlePlot(df_an.iloc[])
cp.show_plot(line_traces=[MA_S, MA_L])

SyntaxError: invalid syntax. Perhaps you forgot a comma? (2429351855.py, line 1)

In [84]:
ic.load_instruments("../data")

In [85]:
ic.instruments_dict[pair]

{'name': 'USD_CHF', 'ins_type': 'CURRENCY', 'displayName': 'USD/CHF', 'pipLocation': 0.0001, 'tradeUnitsPercision': 0, 'marginRate': 0.03}

In [86]:
ins_data = ic.instruments_dict[pair]

In [87]:
df_trades.head()

,time,mid_o,mid_h,mid_l,mid_c,MA_50,MA_200,DELTA,DELTA_PREV,TRADE,DIFF,GAIN,GAIN_C
349,2023-02-20 10:00:00+00:00,0.92380,0.92452,0.92226,0.92408,0.922784,0.922704,0.000080,-0.000016,1,0.00458,45.8,45.8
464,2023-03-17 13:00:00+00:00,0.92792,0.92934,0.92638,0.92866,0.927109,0.927126,-0.000018,0.000073,-1,-0.02678,267.8,313.6
733,2023-05-19 09:00:00+00:00,0.90298,0.90366,0.90040,0.90188,0.895849,0.895780,0.000069,-0.000231,1,-0.00438,-43.8,269.8
865,2023-06-20 09:00:00+00:00,0.89690,0.89840,0.89641,0.89750,0.900077,0.900154,-0.000077,0.000232,-1,-0.01901,190.1,459.9
1107,2023-08-15 17:00:00+00:00,0.87834,0.87890,0.87724,0.87849,0.875709,0.875557,0.000153,-0.000013,1,0.01798,179.8,639.7


In [88]:
df_trades["DIFF"] = df_trades.mid_c.diff().shift(-1)
df_trades.fillna(0, inplace=True)

In [89]:
df_trades["GAIN"] = df_trades["DIFF"] / ins_data.pipLocation
df_trades["GAIN"] = df_trades["GAIN"] * df_trades["TRADE"]

In [90]:
df_trades.GAIN.sum()

np.float64(1632.199999999999)

In [91]:
df_trades["GAIN_C"] = df_trades["GAIN"].cumsum()

In [92]:
cp = CandlePlot(df_trades, candles=False)
cp.show_plot(line_traces=["GAIN_C"])